In [1]:
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader  # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Importing text splitter from Langchain

from langchain_openai import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document  # Importing Document schema from Langchain
from langchain_chroma import Chroma  # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI

import os  # Importing os module for operating system functionalities
import shutil  # Importing shutil module for high-level file operations

In [2]:
import os

# Set the environment variable
os.environ['OPENAI_API_KEY'] = 

# Verify that the environment variable is set
print(os.environ['OPENAI_API_KEY'])

sk-QX2WIwMC3mrFd3nN6PKa1ixf8jbggqFN0_MEa77QscT3BlbkFJKyseiOomj_PaASe5L8gkKJ-tdslbAznXpOVAzQsNoA


In [6]:
CHROMA_PATH = "Basic_chroma"
DATA_PATH = "../preprocesing/output_json_files/"

In [7]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
    
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

In [8]:
import os
import json
from langchain.schema import Document

def load_documents(DATA_PATH):
    """
    Load text documents from JSON files in the specified directory.

    Returns:
        List of Document objects: Loaded text documents represented as Langchain Document objects.
    """
    documents = []

    # Iterate over each file in the directory
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".json"):
            file_path = os.path.join(DATA_PATH, filename)
            
            # Open and read each JSON file
            with open(file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                
                # Extract book, author, and stories from JSON
                book_name = data.get("book", "")
                author = data.get("author", "")
                stories = data.get("stories", [])

                # Iterate through each story and create Document objects
                for story in stories:
                    if "story" in story or "author" in story:
                        story_title = story.get("story") or story.get("author")
                        story_content = story.get("content", "")

                        # Append the story title to the beginning of the story content
                        combined_content = f"{story_title}: {story_content}"

                        # Create a Document object with metadata
                        document = Document(
                            page_content=combined_content,
                            metadata={
                                "book": book_name,
                                "author": author,
                                "story": story_title
                            }
                        )
                        # Append the document to the list of documents
                        documents.append(document)

    return documents

# Example usage:

documents = load_documents(DATA_PATH)

# Initialize a counter
counter = 0

# Loop through the documents and print the metadata and first 10 characters of content
for doc in documents:
    print(doc.metadata, doc.page_content[:100])  # Print metadata and first 10 characters of content
    counter += 1
    if counter >= 10:
        break


{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'ангел каралийчев'} ангел каралийчев: в тази книга са събрани едни от най-добрите приказки от ненадминатия български раз
{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'царят с , магарешките уши'} царят с , магарешките уши: цар траян имал магарешки уши, но ги криел под короната си. никой не знаел
{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'еднооката'} еднооката: имало някога една несвястна жена. мъжът й се трудел от сутрин до вечер, а тя се щурала на
{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'хитрият кръчмар'} хитрият кръчмар: един хитрец отворил кръчма в някакъв град отвъд морето. от всички страни пристигали
{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'сърната и лозата'} сърната и лозата: цял ден ловците преследвали една сърна. надвечер подгоненото животно изс

In [9]:
import os
import json
from langchain.schema import Document
from collections import defaultdict

def load_documents(DATA_PATH):
    """
    Load text documents from JSON files in the specified directory.

    Returns:
        List of Document objects: Loaded text documents represented as Langchain Document objects.
    """
    documents = []

    # Iterate over each file in the directory
    for filename in os.listdir(DATA_PATH):
        if filename.endswith(".json"):
            file_path = os.path.join(DATA_PATH, filename)
            
            # Open and read each JSON file
            with open(file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                
                # Extract book, author, and stories from JSON
                book_name = data.get("book", "")
                author = data.get("author", "")
                stories = data.get("stories", [])

                # Iterate through each story and create Document objects
                for story in stories:
                    if "story" in story or "author" in story:
                        story_title = story.get("story") or story.get("author")
                        story_content = story.get("content", "")

                        # Append the story title to the beginning of the story content
                        combined_content = f"{story_title}: {story_content}"

                        # Create a Document object with metadata
                        document = Document(
                            page_content=combined_content,
                            metadata={
                                "book": book_name,
                                "author": author,
                                "story": story_title
                            }
                        )
                        # Append the document to the list of documents
                        documents.append(document)

    return documents

def find_duplicate_documents(documents):
    """
    Find duplicate documents based on combined content.

    Args:
        documents (list): List of Document objects.

    Returns:
        Dict: Dictionary containing duplicated content and corresponding metadata.
    """
    content_tracker = defaultdict(list)

    # Track each document's combined content and its metadata
    for doc in documents:
        content = doc.page_content.strip().lower()  # Normalize for comparison
        content_tracker[content].append(doc)

    # Find duplicates
    duplicates = {content: docs for content, docs in content_tracker.items() if len(docs) > 1}

    # Print the duplicated documents
    for content, docs in duplicates.items():
        print(f"Duplicated content: '{content[:100]}...' found in the following documents:")
        for doc in docs:
            print(f"  - Book: {doc.metadata['book']}, Author: {doc.metadata['author']}, Story: {doc.metadata['story']}")

    return duplicates

def keep_unique_documents(documents):
    """
    Keep only one unique record for each duplicate, removing redundant records.

    Args:
        documents (list): List of Document objects.

    Returns:
        List of Document objects: List with unique documents, retaining one instance for each duplicate.
    """
    content_tracker = {}
    unique_documents = []

    # Track each document's combined content and keep only the first occurrence
    for doc in documents:
        content = doc.page_content.strip().lower()  # Normalize for comparison
        if content not in content_tracker:
            # Add the first occurrence to the unique list
            content_tracker[content] = doc
            unique_documents.append(doc)

    return unique_documents

# Find and print duplicates
#duplicates = find_duplicate_documents(documents)

# Keep only unique documents
unique_documents = keep_unique_documents(documents)

# Print the unique documents count
print(f"\nTotal unique documents retained: {len(unique_documents)}")



Total unique documents retained: 847


In [10]:
def split_text(documents: list[Document]):
    """
    Split the text content of the given list of Document objects into smaller chunks.

    Args:
        documents (list[Document]): List of Document objects containing text content to split.

    Returns:
        list[Document]: List of Document objects representing the split text chunks.
    """
    # Initialize text splitter with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,  # Size of each chunk in characters
        chunk_overlap=200,  # Overlap between consecutive chunks
        length_function=len,  # Function to compute the length of the text
        add_start_index=True,  # Flag to add start index to each chunk
    )
    # Split documents into smaller chunks using text splitter
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    # Print example of page content and metadata for a chunk
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks  # Return the list of split text chunks

In [11]:

def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    
    # print(chunks)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, embeddings, persist_directory=CHROMA_PATH
    )
    #db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


def generate_data_store():
    """
    Function to generate vector database in chroma from documents.
    """
    documents = load_documents(DATA_PATH)  # Load documents from a source
    unique_documents = keep_unique_documents(documents)  # Keep only unique documents
    chunks = split_text(unique_documents)  # Split documents into manageable chunks
    save_to_chroma(chunks)  # Save the processed data to a data store    

# Generate the data store
generate_data_store()

Split 847 documents into 5542 chunks.
когато мъжът отишъл на нивата, в селото влязъл грънчар с кола грънци и почнал да вика — грънци продавам, грънци! излязла еднооката и попитала — как ги даваш? — пълно за пълно. жената грабнала едно кринче и влязла в житницата да го напълни със зърно. като гребяла, тя напипала заровеното гърне и си рекла — ех, че съм късметлийка! намерих си просото. ще дам на грънчаря просо наместо жито. напълнила кринчето с жълтици и го отнесла. като видял парите, на грънчаря му светнали очите, той поел кринчето, изсипал всичките жълтици в торбата си и побягнал, като оставил и колата, и биволите, и грънците. — брей, че глупав грънчар! — засмяла се еднооката и смъкнала от колата всичките грънци. наредила ги по колците на плета. за всичките грънци имало място, само за едно пукнато гърне нямало. — сторете му
{'book': 'Български народни приказки ак', 'author': 'ангел каралийчев', 'story': 'еднооката', 'start_index': 3003}
Saved 5542 chunks to Basic_chroma.
